In [1]:
#Travelling Salesman

In [2]:
import genetic
import datetime
import numpy as np
import copy
from functools import reduce
import random
import time
from math import sqrt, exp
import math

In [3]:
#Define the genes
class Location:
    def __init__(self, name, x, y):
        self.name = name
        self.x = x
        self.y = y

A = Location('A', 4, 7)
B = Location('B', 2, 6)
C = Location('C', 0, 5)
D = Location('D', 1, 3)
E = Location('E', 3, 0)
F = Location('F', 5, 1)
G = Location('G', 7, 2)
H = Location('H', 6, 4)

genes = [A, B, C, D, E, F, G, H]

In [4]:
#Define the individual
        
class Path(genetic.Individual):    
    def __init__(self, genome, fitness, age=0):
        self.genome = genome
        self.fitness = fitness
        self.age = 0

In [5]:
#Define the initialization

def initialize(lambda_, genes, individual, fitness):
    init_pop = []
    for n in range(lambda_):
        new_genes = copy.copy(genes)
        random.shuffle(new_genes)
        fit = fitness(new_genes)
        init_pop.append(individual(new_genes, fit))
    return init_pop

In [6]:
#Define the fitness

class Fitness:
    def __init__(self, distance):
        self.total_distance = distance
        
    def __ne__(self, other):
        return self.total_distance != other.total_distance
                       
    def __gt__(self, other):
        return self.total_distance < other.total_distance
    
    def __lt__(self, other):
        if other == np.inf:
            return True

        
def get_fitness(genes):
    dist = 0
    for i in range(len(genes)-1):
        l1 = genes[i]
        l2 = genes[i+1]
        dist += sqrt(((l1.x-l2.x)**2)+((l1.y-l2.y)**2))
    l1 = genes[-1]
    l2 = genes[0]
    dist += sqrt(((l1.x-l2.x)**2)+((l1.y-l2.y)**2))
    return Fitness(dist)
        
    

In [7]:
#define mutation

def swap(genes):
    new_genes = genes
    index = random.sample(list(range(len(new_genes))), 2)
    swap = new_genes[index[0]]
    new_genes[index[0]] = new_genes[index[1]]
    new_genes[index[1]] = swap
    return new_genes

def mutate(ind, gen, fitness, ind_class):
    genome = copy.copy(ind.genome)
    if random.random() < .8:
        genome = swap(genome)
    else:
        for n in range(random.randrange(1, len(ind.genome)//2)):
            genome = swap(genome)
    fit = fitness(genome)
    return ind_class(genome, fit)

In [8]:
#Define sampling

def truncation_selection(population, n, gen, fitness, anneal=False):
    population.sort(key=lambda x: x.fitness, reverse=True)
    if anneal is True:
        individual = population[n-1]
        test = population[n]
        if exp(-(individual-population[n])/(1/log(gen))) > random.random():
            population[n-1] = test
            population[n] = individual
    return population[:n]

def tournament_selection(population, n, gen, fitness, p=1.8):
    winners = []
    for i in range(n):
        if p%1 != 0:
            if random.random() < p%1:
                loop_p = math.ceil(p)
            else:
                loop_p = math.floor(p)
        else:
            loop_p = p
        contestants = random.sample(population, loop_p)
        contestants.sort(key=lambda x: x.fitness, reverse=True)
        winners.append(contestants[0])
    return winners


In [9]:
#Define display

def display(ind, start_time):
    print(time.time()-start_time)
    path = [x.name for x in ind.genome]
    print(f'The individual\'s genome: \n {path} \n and fitness: {ind.fitness.total_distance}')

In [10]:
tsp = genetic.EvFuncs(genes, Path, initialize, get_fitness,
            mutate, truncation_selection, display)

In [11]:
tsp_es = genetic.ES(tsp, 1, 1, timer=10)  

In [12]:
a = tsp_es.solve()

0.00012993812561035156
The individual's genome: 
 ['G', 'F', 'D', 'A', 'E', 'C', 'B', 'H'] 
 and fitness: 33.55449554920931
0.00031185150146484375
The individual's genome: 
 ['D', 'F', 'G', 'H', 'E', 'C', 'B', 'A'] 
 and fitness: 29.247359759844038
0.0003876686096191406
The individual's genome: 
 ['H', 'F', 'G', 'D', 'E', 'C', 'B', 'A'] 
 and fitness: 28.995298568739248
0.0006368160247802734
The individual's genome: 
 ['H', 'G', 'F', 'E', 'D', 'A', 'C', 'B'] 
 and fitness: 26.49409509546231
0.002816915512084961
The individual's genome: 
 ['F', 'E', 'D', 'B', 'C', 'A', 'H', 'G'] 
 and fitness: 23.789788076095096
0.0061228275299072266
The individual's genome: 
 ['H', 'A', 'B', 'C', 'D', 'E', 'F', 'G'] 
 and fitness: 20.627510415926718
0.050122976303100586
The individual's genome: 
 ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H'] 
 and fitness: 20.627510415926714


In [13]:
import re
points = '''1 38.24 20.42
2 39.57 26.15
3 40.56 25.32
4 36.26 23.12
5 33.48 10.54
6 37.56 12.19
7 38.42 13.11
8 37.52 20.44
9 41.23 9.10
10 41.17 13.05
11 36.08 -5.21
12 38.47 15.13
13 38.15 15.35
14 37.51 15.17
15 35.49 14.32
16 39.36 19.56
'''
points = re.split("[' ', '\n']", points)
genes = []
for i in range(0, len(points)-1, 3):
    genes.append(Location(points[i], float(points[i+1]), float(points[i+2])))

In [16]:
tsp = genetic.EvFuncs(genes, Path, initialize, get_fitness,
            mutate, tournament_selection, display)
tsp_es = genetic.ES(tsp, 2, 2, timer=300)  

In [17]:
a = tsp_es.solve()

0.00017714500427246094
The individual's genome: 
 ['4', '2', '14', '9', '11', '13', '8', '6', '10', '7', '12', '15', '16', '3', '5', '1'] 
 and fitness: 126.70117510254043
0.00040721893310546875
The individual's genome: 
 ['14', '6', '9', '3', '2', '11', '10', '13', '4', '1', '8', '15', '16', '12', '5', '7'] 
 and fitness: 123.75578337684152
0.001373291015625
The individual's genome: 
 ['4', '8', '10', '12', '5', '6', '1', '3', '16', '11', '7', '9', '13', '14', '15', '2'] 
 and fitness: 120.5330653655846
0.003748178482055664
The individual's genome: 
 ['8', '3', '10', '4', '16', '2', '13', '7', '12', '11', '9', '14', '6', '5', '15', '1'] 
 and fitness: 117.60329949709607
0.004145145416259766
The individual's genome: 
 ['1', '2', '3', '4', '7', '8', '13', '12', '16', '11', '9', '14', '6', '5', '15', '10'] 
 and fitness: 112.39334153699204
0.005105257034301758
The individual's genome: 
 ['1', '2', '4', '8', '7', '10', '13', '5', '15', '6', '12', '9', '11', '14', '16', '3'] 
 and fitness:

In [ ]:
index = [12, 16, 1, 3, 2, 4, 8, 15, 5, 11, 9, 10, 7, 6, 14, 13]
gen = [genes[i-1] for i in index]

In [ ]:
get_fitness(gen).total_distance

In [ ]:
index_2 =  ['11', '5', '6', '9', '10', '12', '14', '13', '15', '7', '8', '16', '1', '4', '3', '2'] 
index = [int(x) for x in index_2]
gen = [genes[i-1] for i in index]
get_fitness(gen).total_distance

In [ ]:
a = 5

In [ ]:
print('asdf', a)